In [ ]:
################################################################
#
# Zillow Data Exploration: Wicker Park, Bucktown, Logan Square
# Developer: Marcus Bischof
#
################################################################

import pandas as pd
import numpy as np

# Vizualization
import seaborn as sns
from matplotlib import pyplot as plt

# API
import http.client
# Zillow's API returns XML
import xml.etree.ElementTree as ET

# Utility
import collections
import os
import re

zillow_api_key = os.environ['ZILLOW_API_KEY']

In [ ]:
################################################################
#
# Functions that we may re-use in this exploration.
#
################################################################

# Code used from: https://stackoverflow.com/a/10076823/6169225
# StackOverflow user who developed this: K3---rnc
def etree_to_dict(t):
    """ This returns a dictionary from an object (= ET.fromstring(response.read().decode())). """
    d = {t.tag: {} if t.attrib else None}
    children = list(t)
    if children:
        dd = collections.defaultdict(list)
        for dc in map(etree_to_dict, children):
            for k, v in dc.items():
                dd[k].append(v)
        d = {t.tag: {k: v[0] if len(v) == 1 else v
                     for k, v in dd.items()}}
    if t.attrib:
        d[t.tag].update(('@' + k, v)
                        for k, v in t.attrib.items())
    if t.text:
        text = t.text.strip()
        if children or t.attrib:
            if text:
              d[t.tag]['#text'] = text
        else:
            d[t.tag] = text
    return d

In [ ]:
# Establish connection with zillow
connection = http.client.HTTPSConnection('www.zillow.com')

# Headers & payload for the connection.request(s) that will be called below
headers = {'Content-type': 'application/json'}
payload = ""

# API call to understand the neighborhoods in Chicago that Zillow has data on
# Fingers crossed we get the neighborhoods I'm curious about: Wicker Park, Bucktown, Logan Square
genre_request = '/webservice/GetRegionChildren.htm?zws-id={}&state=il&city=chicago&childtype=neighborhood'.format(zillow_api_key)

try:
    connection.request('GET', genre_request, payload, headers)
except e:    # This is the correct syntax
    print('There was an error making the request to Zillow\'s API.')
    print(e)

response = connection.getresponse()

# Load XML data into tree obj
tree = ET.fromstring(response.read().decode())

In [ ]:
# Let's find our neighborhoods
initial_dict_obj = etree_to_dict(tree)
chicago_neighborhoods = pd.DataFrame(initial_dict_obj["".join([key for key in initial_dict_obj.keys()])]['response']['list']['region'])

In [ ]:
chicago_neighborhoods['zindex'] = pd.Series([
    int(str(val).split('{\'@currency\': \'USD\', \'#text\': \'')[1][:-2]) if len(str(val)) > 3 
    else None 
    for val in chicago_neighborhoods['zindex']
])

In [ ]:
# Let's look!
chicago_neighborhoods[chicago_neighborhoods.name.isin(['Logan Square', 'Bucktown', 'Wicker Park'])]

In [ ]:
connection.close()